# Tree-Based Models for Asset Pricing: Simulation Study

## Overview
This notebook implements and evaluates three tree-based machine learning models for predicting asset returns:
1. **Random Forest (RF)**: Ensemble of decision trees with bagging and feature randomization
2. **Gradient Boosted Regression Trees (GBRT)**: Sequential ensemble with gradient descent optimization
3. **GBRT with Huber Loss**: Robust version using Huber loss function

## Key Concepts
- **R²**: Measures predictive power (1 - SSE/SST)
- **Hyperparameter Tuning**: Grid search using test set, final evaluation on out-of-sample set
- **Feature Importance**: Quantifies the contribution of each characteristic to predictions
- **VIP (Variable Importance)**: Measures impact on prediction accuracy

# Part 0: Setup

Import necessary libraries and define helper functions.

**Important**: Make sure all imports run successfully before continuing.

In [ ]:
import numpy as np
import pandas as pd
import os

%matplotlib inline

# Auto-reload modules when they change (useful for development)
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ 
    Calculate relative error between x and y.
    Useful for checking if your implementation matches expected values.
    
    Returns: max |x - y| / (|x| + |y|)
    A value < 1e-7 indicates high accuracy.
    """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# Data Generation

## Understanding the Data Generation Process (DGP)

The `dgp()` function generates simulated data with the following structure:

### Panel Structure
- **N = 200** firms (cross-sectional units)
- **T = 180** time periods (e.g., months)
- **P characteristics** (features)
  - P=100 → 50 characteristics (2P features due to transformations)
  - P=200 → 100 characteristics

### Output Files (in ./Simu/SimuData_{P}/)
1. **c{M}.csv**: Firm characteristics matrix [N×T rows, P columns]
   - Each row represents a firm-time observation
   - Characteristics are standardized and may include nonlinear transformations

2. **r{model}_{M}_{hh}.csv**: Returns vector [N×T rows]
   - model: Different data generating processes (typically 1)
   - M: Monte Carlo simulation number
   - hh: Return horizon
     - 1 = monthly returns
     - 3 = quarterly (3-month) returns
     - 6 = semi-annual returns
     - 12 = annual returns

### Parameters
- **P**: Number of characteristics (100 or 200)
- **hh**: List of return horizons to generate

**Note**: You don't need to modify DGP.py, but read it to understand the data structure!

In [ ]:
from DGP import dgp

# Set data generation parameters
P = 100        # Number of base characteristics (50 actual features for P=100)
hh = [1]       # Return horizon: [1] for monthly returns

# Generate simulated data
# This creates ./Simu/SimuData_100/ directory with characteristics and returns
dgp(P, hh)

# After running, verify the data files exist:
# - ./Simu/SimuData_100/c1.csv (characteristics)
# - ./Simu/SimuData_100/r1_1_1.csv (returns for model 1, MC 1, horizon 1)

# Part 1: Implementing Tree Models

## Background: Tree-Based Models in Asset Pricing

### Why Trees?
Traditional linear models assume:
$$r_{i,t+1} = \beta_0 + \sum_{j=1}^P \beta_j z_{ij,t} + \epsilon_{i,t+1}$$

But this misses:
1. **Nonlinear relationships**: The effect of size might differ at different levels
2. **Interactions**: Small growth stocks might behave differently than large growth stocks

### Regression Trees
A tree model partitions the feature space into rectangular regions:
$$g(z_{i,t}) = \sum_{k=1}^{K} \theta_k \mathbb{1}\{z_{i,t} \in C_k\}$$

where:
- $K$ = number of leaf nodes (terminal regions)
- $C_k$ = partition $k$ (defined by a sequence of splits)
- $\theta_k$ = average return in partition $k$

**Example Split Logic:**
```
If size < 1000:
    If growth > 0.5: predict θ₁ = 0.12  (small growth stocks)
    Else: predict θ₂ = 0.05             (small value stocks)
Else:
    If growth > 0.5: predict θ₃ = 0.08  (large growth stocks)
    Else: predict θ₄ = 0.06             (large value stocks)
```

### Random Forests
Single trees overfit. Random forests average many trees trained on:
1. **Bootstrap samples** of the data (bagging)
2. **Random subsets** of features at each split (decorrelation)

Final prediction:
$$\hat{r}_{i,t+1} = \frac{1}{B} \sum_{b=1}^B g_b(z_{i,t})$$

### Gradient Boosted Regression Trees (GBRT)
Instead of averaging, boosting builds trees sequentially:
1. Fit tree $g_1$ to returns
2. Fit tree $g_2$ to residuals from $g_1$
3. Fit tree $g_3$ to residuals from $g_1 + \nu g_2$
4. Continue for $B$ iterations

Final model:
$$\hat{r}_{i,t+1} = g_1(z_{i,t}) + \nu g_2(z_{i,t}) + \nu g_3(z_{i,t}) + \cdots + \nu g_B(z_{i,t})$$

where $\nu$ is the learning rate (shrinkage parameter).

**Key hyperparameters:**
- Learning rate $\nu$: Lower values need more trees but generalize better
- Number of trees $B$: Selected via validation to prevent overfitting
- Tree depth $L$: Controls complexity (usually kept shallow, e.g., 2)

### Huber Loss (Robust GBRT)
Standard GBRT uses squared error loss:
$$L(y, \hat{y}) = (y - \hat{y})^2$$

Huber loss is more robust to outliers:
$$L_\delta(y, \hat{y}) = \begin{cases}
\frac{1}{2}(y - \hat{y})^2 & \text{if } |y - \hat{y}| \leq \delta \\
\delta |y - \hat{y}| - \frac{1}{2}\delta^2 & \text{otherwise}
\end{cases}$$

This is quadratic for small errors (like squared loss) but linear for large errors (less sensitive to outliers).

## Implementation Guide

### Task 1: Implement Random Forest

Open `Tree.py` and complete the `Random_forest()` function.

**Your implementation should:**
1. Loop through hyperparameter combinations (max_features × max_depth)
2. For each combination:
   - Train a `RandomForestRegressor`
   - Compute R² on test, OOS, and training sets
3. Select the model with best test R²
4. Return OOS R², IS R², feature importances, and VIP

**R² Formula:**
$$R^2 = 1 - \frac{\sum(y_i - \hat{y}_i)^2}{\sum(y_i - \bar{y}_{\text{train}})^2}$$

where $\bar{y}_{\text{train}}$ is the mean of training returns (benchmark).

**Key sklearn methods:**
```python
clf = RandomForestRegressor(max_depth=..., n_estimators=..., max_features=...)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
importances = clf.feature_importances_  # Array of importance scores
```

In [ ]:
# Test your Random Forest implementation
from Tree import Random_forest

# Set random seed for reproducibility
np.random.seed(1)

# Create synthetic test data
nump = 50              # Dataset size indicator
ne = 30                # Number of trees
N = 1000               # Number of observations
p = 200                # Number of features

# Generate random data
xtrain = np.random.randn(N, p)
ytrain = np.random.randn(N)
mtrain = np.mean(ytrain)

xoos = np.random.randn(N, p)
yoos = np.random.randn(N)

xtest = np.random.randn(N, p)
ytest = np.random.randn(N)

# Run your implementation
r2_oos, r2_is, b, v = Random_forest(nump, ne, xtrain, ytrain, mtrain, xoos, yoos, xtest, ytest)

# Expected values (from reference implementation)
expect_roos = -0.0051077874136
expect_ris = 0.24486374236437

# Check if your implementation is correct
# Relative error should be < 1e-7
print(f"OOS R² relative error: {rel_error(r2_oos, expect_roos):.2e}")
print(f"IS R² relative error: {rel_error(r2_is, expect_ris):.2e}")

if rel_error(r2_oos, expect_roos) < 1e-7 and rel_error(r2_is, expect_ris) < 1e-7:
    print("✓ Random Forest implementation is CORRECT!")
else:
    print("✗ Check your implementation - errors are too large")

### Task 2: Implement GBRT

Complete the `GBRT()` function in `Tree.py`.

**Your implementation should:**
1. Loop through learning rates
2. For each learning rate:
   - Train a `GradientBoostingRegressor`
   - Use `staged_predict()` to get predictions at each boosting iteration
   - Compute R² at each stage for test/OOS/training sets
3. Select best (learning_rate, n_estimators) based on test R²
4. Retrain with optimal hyperparameters and return metrics

**Key differences from Random Forest:**
- `staged_predict()` returns a generator of predictions (one per tree added)
- Need to loop through iterations to evaluate at each stage
- Learning rate controls contribution of each tree

**sklearn methods:**
```python
clf = GradientBoostingRegressor(max_depth=2, 
                                 n_estimators=50,
                                 learning_rate=0.1,
                                 loss='squared_error')  # or 'huber'
clf.fit(X_train, y_train)

# Get predictions at each boosting stage
for i, pred in enumerate(clf.staged_predict(X_test)):
    # pred contains predictions using first (i+1) trees
    r2[i] = compute_r2(pred, y_test)
```

In [ ]:
# Test your GBRT implementation with Huber loss
from Tree import GBRT

np.random.seed(2)

# Smaller test case for faster execution
ne = 4
N = 1000
p = 8

xtrain = np.random.randn(N, p)
ytrain = np.random.randn(N)
mtrain = np.mean(ytrain)

xoos = np.random.randn(N, p)
yoos = np.random.randn(N)

xtest = np.random.randn(N, p)
ytest = np.random.randn(N)

# Learning rate grid: 10^(-3) and 10^(-2)
lamv = [-3, -2]
mtype = 'huber'  # Use robust Huber loss

r2_oos, r2_is, b, v = GBRT(lamv, mtype, ne, xtrain, ytrain, mtrain, xoos, yoos, xtest, ytest)

# Expected values
expect_roos = -0.00143708500012174
expect_ris = 0.00170909909275196
expect_b = [0.40641374, 0.28890748, 0, 0, 0, 0, 0.30467878, 0]
expect_v = [-0.24905571, -0.07560964, 0, 0, 0, 0, -0.05731088, 0]

# Validate results
print(f"OOS R² error: {rel_error(r2_oos, expect_roos):.2e}")
print(f"IS R² error: {rel_error(r2_is, expect_ris):.2e}")
print(f"Feature importance error: {rel_error(b, expect_b):.2e}")
print(f"VIP error: {rel_error(v, expect_v):.2e}")

if all([rel_error(r2_oos, expect_roos) < 1e-7,
        rel_error(r2_is, expect_ris) < 1e-7,
        rel_error(b, expect_b) < 1e-7,
        rel_error(v, expect_v) < 1e-7]):
    print("✓ GBRT (Huber) implementation is CORRECT!")
else:
    print("✗ Check your implementation")

In [ ]:
# Test GBRT with squared error loss
from Tree import GBRT

np.random.seed(3)

ne = 4
N = 1000
p = 8

xtrain = np.random.randn(N, p)
ytrain = np.random.randn(N)
mtrain = np.mean(ytrain)

xoos = np.random.randn(N, p)
yoos = np.random.randn(N)

xtest = np.random.randn(N, p)
ytest = np.random.randn(N)

lamv = [-3, -2]
mtype = 'ls'  # Least squares loss

r2_oos, r2_is, b, v = GBRT(lamv, mtype, ne, xtrain, ytrain, mtrain, xoos, yoos, xtest, ytest)

expect_roos = -3.3197655167960e-05
expect_ris = 0.0015452687210351
expect_b = [0.02997661, 0.01498528, 0, 0.5588082, 0, 0, 0, 0.39622991]
expect_v = [0.00265778, 0.00110131, 0, -0.26560026, 0, 0, 0, -0.08877995]

print(f"OOS R² error: {rel_error(r2_oos, expect_roos):.2e}")
print(f"IS R² error: {rel_error(r2_is, expect_ris):.2e}")
print(f"Feature importance error: {rel_error(b, expect_b):.2e}")
print(f"VIP error: {rel_error(v, expect_v):.2e}")

if all([rel_error(r2_oos, expect_roos) < 1e-7,
        rel_error(r2_is, expect_ris) < 1e-7,
        rel_error(b, expect_b) < 1e-7,
        rel_error(v, expect_v) < 1e-7]):
    print("✓ GBRT (Least Squares) implementation is CORRECT!")
else:
    print("✗ Check your implementation")

# Part 2: Monte Carlo Simulation

## Understanding the Simulation

Now that your implementations are working, run the full simulation.

### What happens in MC_tree()?

1. **Data Loading**: Reads characteristics and returns from DGP output
2. **Data Splitting**:
   - Training: First 60 months (T/3 = 180/3 = 60)
   - Test: Next 60 months (for hyperparameter tuning)
   - OOS: Final 60 months (for final evaluation)
3. **Model Training**: Trains all three models
4. **Results Saved**:
   - R² scores: `roos_{model}_{MC}.csv` and `ris_{model}_{MC}.csv`
   - Feature importances: `B/b{model}_{MC}_{method}.csv`
   - VIP scores: `VIP/b{model}_{MC}_{method}.csv`

### Interpreting Results

**R² Values:**
- Positive R²: Model predicts better than the mean
- Negative R²: Model performs worse than predicting the mean
- IS R² > OOS R²: Expected due to overfitting
- If OOS R² >> IS R²: Likely a bug in your code!

**Feature Importances:**
- Sum to 1.0
- Larger values → more important characteristics
- Can identify which firm characteristics drive returns

**Comparing Models:**
- Random Forest: Often highest in-sample R² (prone to overfitting)
- GBRT: Usually better out-of-sample performance
- GBRT+Huber: Most robust to outliers

### Parameters to Experiment With

```python
MC = [1, 2]        # Run 2 simulations (different random seeds)
datanum = 100      # Use P=100 dataset (50 characteristics)
horizon = [1]      # Monthly returns
mo = [1]           # Model type from DGP
```

**Note**: The simulation takes ~15-20 minutes per Monte Carlo iteration.
Start with MC=[1] to verify everything works!

In [ ]:
from Tree import MC_tree

# Simulation parameters
MC = [1, 1]        # Run two iterations (you can increase this for more robust results)
datanum = 100      # Dataset size (50 characteristics)
horizon = [1]      # Monthly return horizon
mo = [1]           # Model specification

# Run the simulation
# This will:
# 1. Load data from ./Simu/SimuData_100/
# 2. Train Random Forest, GBRT, and GBRT+Huber
# 3. Save results to ./Simu/Simu_100/Tree1/
print("Starting Monte Carlo simulation...")
print("This may take 15-20 minutes per iteration.")
print()

MC_tree(MC, datanum, horizon, mo)

print()
print("Simulation complete!")
print("Results saved to:", f"./Simu/Simu_{datanum}/Tree{horizon[0]}/")

## Analyzing Results

After the simulation completes, examine the output files:

In [ ]:
# Load and display R² results
import pandas as pd

# Path to results
results_dir = f"./Simu/Simu_{datanum}/Tree{horizon[0]}/"

# Read OOS R² for first MC iteration
r2_oos = pd.read_csv(results_dir + 'roos_1_1.csv', header=None, names=['R2_OOS'])
r2_is = pd.read_csv(results_dir + 'ris_1_1.csv', header=None, names=['R2_IS'])

# Combine into a single DataFrame
results = pd.DataFrame({
    'Model': ['Random Forest', 'GBRT (LS)', 'GBRT (Huber)'],
    'OOS R²': r2_oos['R2_OOS'].values,
    'IS R²': r2_is['R2_IS'].values
})

print("\nPredictive Performance Comparison:")
print("=" * 50)
print(results.to_string(index=False))
print()

# Insights
print("Interpretation:")
print(f"- Best OOS performance: {results.loc[results['OOS R²'].idxmax(), 'Model']}")
print(f"- Least overfitting: {results.loc[(results['IS R²'] - results['OOS R²']).idxmin(), 'Model']}")